In [4]:
import numpy as np
import os
import sys 
import time
from collections import deque
from multiprocessing.dummy import Pool
from numpy import linalg as LA
import gc


In [5]:

def combine_centroid(I, label_cent, threads = 12):
    indexes = np.argsort(I)
    unique, counts = np.unique(I, return_counts = True)
    uc = dict(zip(unique, counts))
    uc[-1] = 0
    for i in range(label_cent.shape[0]):
        if i not in uc:
            uc[i] = 0
        uc[i] = uc[i-1] + uc[i]

    def update(i):
        cur_index = indexes[uc[i-1]:uc[i]]
        I[cur_index] = label_cent[i]

    p = Pool(threads)
    p.map_async(update, range(label_cent.shape[0])).get(60*60*24*356)

    return I.astype(np.int)


In [8]:
combine_centroid(np.array([1,3,2,1]), np.array([1,3,2]))

array([3, 3, 2, 3])

In [15]:
indexes = np.argsort(np.array([1,3,2,1]))

unique, counts = np.unique(np.array([1,3,2,1]), return_counts = True)
indexes,unique, counts

(array([0, 3, 2, 1]), array([1, 2, 3]), array([2, 1, 1]))

In [17]:
uc = dict(zip(unique, counts))
uc

{1: 2, 2: 1, 3: 1}

In [19]:
np.array([1,3,2]).shape[0]

3

In [20]:
import faiss

In [21]:
import numpy as np
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
xb[:, 0] += np.arange(nb) / 1000.
xq = np.random.random((nq, d)).astype('float32')
xq[:, 0] += np.arange(nq) / 1000.

In [22]:
import faiss                   # make faiss available
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

True
100000


In [24]:
k = 4                          # we want to see 4 nearest neighbors
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)

[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]
[[0.        7.1751733 7.207629  7.2511625]
 [0.        6.3235645 6.684581  6.7999454]
 [0.        5.7964087 6.391736  7.2815123]
 [0.        7.2779055 7.5279865 7.6628466]
 [0.        6.7638035 7.2951202 7.3688145]]


In [38]:
D, I = index.search(xq, k)     # actual search
print(I[:5])                   # neighbors of the 5 first queries
print(I[-5:])                  # neighbors of the 5 last queries

[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
[[ 9900 10500  9309  9831]
 [11055 10812 11321 10260]
 [11353 10164  9787 10719]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]


In [27]:
nlist = 100
k = 4
quantizer = faiss.IndexFlatL2(d)  # the other index
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

In [28]:
assert not index.is_trained
index.train(xb)
assert index.is_trained

In [29]:
index.add(xb)                  # add may be a bit slower as well
D, I = index.search(xq, k)     # actual search
print(I[-5:])

[[ 9900 10500  9831 10808]
 [11055 10812 11321 10260]
 [11353 10164 10719 11013]
 [10571 10203 10793 10952]
 [ 9582 10304  9622  9229]]


In [37]:
index.nprobe = 2           # default nprobe is 1, try a few more
D, I = index.search(xq, k)
print(I[-5:])                  # neighbors of the 5 last queries

[[ 9900 10500  9309  9831]
 [11055 10812 11321 10260]
 [11353 10164  9787 10719]
 [10571 10664 10632  9638]
 [ 9628  9554 10036  9582]]


In [39]:
nlist = 100
m = 8                             # number of bytes per vector
k = 4
quantizer = faiss.IndexFlatL2(d)  # this remains the same
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8)
                                    # 8 specifies that each sub-vector is encoded as 8 bits
index.train(xb)
index.add(xb)
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)

[[   0  424  363  278]
 [   1  555 1063   24]
 [   2  304   46  346]
 [   3  773  182 1529]
 [   4  288  754  531]]
[[1.4556826 6.031368  6.18729   6.388527 ]
 [1.4934082 5.742547  6.199413  6.2150173]
 [1.6027994 6.2017474 6.3279257 6.785414 ]
 [1.698049  6.262315  6.269568  6.5604277]
 [1.3023579 6.1362486 6.338999  6.5144215]]


In [45]:
nlist = 100
m = 32                             # number of bytes per vector
k = 4
quantizer = faiss.IndexFlatL2(d)  # this remains the same
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8)
                                    # 8 specifies that each sub-vector is encoded as 8 bits
index.train(xb)
index.add(xb)
D, I = index.search(xb[:5], k) # sanity check
print(I)
print(D)

[[  0 363 100 393]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173 182  18]
 [  4 288 531 178]]
[[0.0219759  7.1710577  7.2605004  7.2859807 ]
 [0.02429624 6.316854   6.557459   6.7429657 ]
 [0.0579994  5.8492284  6.2730465  7.3338146 ]
 [0.03550963 7.23396    7.5488653  7.557202  ]
 [0.03332859 6.5410366  7.203384   7.3425765 ]]
